In [1]:
from tqdm import tqdm
import pandas as pd
import os
import sys

import numpy as np
import re

In [69]:
# basepath = r'C:\Users\xujun\github_repos\data\aclImdb'
# labels = {'pos': 1, 'neg': 0}
# sample_num = 12500
# df = pd.DataFrame()

# for s in ('test', 'train'):

#     for l in ('pos', 'neg'):
#         path = os.path.join(basepath, s, l)
#         sorted_files = sorted(os.listdir(path))

#         for i in tqdm(range(sample_num)):
#             file = sorted_files[i]
#             with open(os.path.join(path, file), 'r', encoding='utf-8') as infile:
#                 txt = infile.read()

#             df = pd.concat([df, pd.DataFrame([[txt, labels[l]]])], ignore_index=True)

# df.columns = ['review', 'sentiment']

100%|██████████| 12500/12500 [00:09<00:00, 1377.07it/s]


In [70]:
# np.random.seed(0)
# df = df.reindex(np.random.permutation(df.index))
# df.to_csv(r'C:\Users\xujun\github_repos\data\movie_data.csv', index=False, encoding='utf-8')

In [2]:
df = pd.read_csv(r'C:\Users\xujun\github_repos\data\movie_data.csv', encoding='utf-8')

In [3]:
df.head(3)

,review,sentiment
0,"In 1974, the teenager Martha Moxley (Maggie Gr...",1
1,OK... so... I really like Kris Kristofferson a...,0
2,"***SPOILER*** Do not read this, if you think a...",0


In [4]:
df.shape

(50000, 2)

In [5]:
df.loc[0, 'review'][-50:]

'is seven.<br /><br />Title (Brazil): Not Available'

In [110]:
def preprocessor(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
    text = (re.sub('[\W]+', ' ', text.lower()) + ' '.join(emoticons).replace('-', ''))
    return text

def tokenizer(text):
    return text.split()

In [107]:
print(preprocessor(df.loc[0, 'review'][-50:]))
print(preprocessor('</a>This :) is :( a test :-)!'))

is seven title brazil not available
this is a test :) :( :)


In [108]:
df['review'] = df['review'].apply(preprocessor)

In [111]:
tokenizer('runners like running and thus they run')

['runners', 'like', 'running', 'and', 'thus', 'they', 'run']

In [112]:
from nltk.stem.porter import PorterStemmer

In [113]:
porter = PorterStemmer()
def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split()]

In [114]:
tokenizer_porter('runners like running and thus they run')

['runner', 'like', 'run', 'and', 'thu', 'they', 'run']

In [6]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

In [7]:
count = CountVectorizer()
docs = np.array(['The sun is shining', 
                 'The weather is sweet',
                 'The sun is shining, the weather is sweet, and one and one is two'])
bag = count.fit_transform(docs)

In [116]:
import nltk
from nltk.corpus import stopwords

In [117]:
stop = stopwords.words('english')
[w for w in tokenizer_porter('a runner likes running and runs a lot') if w not in stop] 

['runner', 'like', 'run', 'run', 'lot']

In [122]:
X_train = df.loc[:25000, 'review'].values
y_train = df.loc[:25000, 'sentiment'].values
X_test = df.loc[:25000, 'review'].values
y_test = df.loc[:25000, 'sentiment'].values

In [82]:
print(count.vocabulary_)
print(bag.toarray())

{'the': 6, 'sun': 4, 'is': 1, 'shining': 3, 'weather': 8, 'sweet': 5, 'and': 0, 'one': 2, 'two': 7}
[[0 1 0 1 1 0 1 0 0]
 [0 1 0 0 0 1 1 0 1]
 [2 3 2 1 1 1 2 1 1]]


In [83]:
from sklearn.feature_extraction.text import TfidfTransformer

In [86]:
tfidf = TfidfTransformer(use_idf=True, 
                         norm='l2',
                         smooth_idf=True)
np.set_printoptions(precision=2)
print(tfidf.fit_transform(count.fit_transform(docs)).toarray())

[[0.   0.43 0.   0.56 0.56 0.   0.43 0.   0.  ]
 [0.   0.43 0.   0.   0.   0.56 0.43 0.   0.56]
 [0.5  0.45 0.5  0.19 0.19 0.19 0.3  0.25 0.19]]


In [91]:
a = [0.,   0.43, 0.,   0.56,0.56, 0.,   0.43, 0.,   0.  ]

In [98]:
np.mean(a)

0.22